## 1. Import Libraries

In [ ]:
import os
import time
import random

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image


In [ ]:
"""In order force cuDNN to onyl use deterministic
algorithms which might be useful to produce
reproducible results we need following code"""

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

## 2. Model settings

In [ ]:
# Setting a random seed
def set_all_seeds(seed):
  os.environ["PL_GLOBAL_SEED"] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

In [ ]:
# Setting cuDNN and PyTorch algorithmic behavior to deterministic
def set_deterministic():
  if torch.cuda_is_available():
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
  torch.set_deterministic(True)

In [ ]:
#### SETTINGS ####

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.0001
BATCH_SIZE = 16
NUM_EPOCHS = 40

# Architecture
NUM_CLASSES=10

#Other
DEVICE = "cuda" if torch.cuda.is_available else "cpu"

set_all_seeds(RANDOM_SEED)

In [ ]:
# Deterministic behavior not yet supported by AdaptiveAvgPool2d
#set_deterministic()


## 3. Import utility functions

In [ ]:
import sys

# In order to import modules from a directory one level up.

sys.path.append('/content/helper_functions/helper_data.py')
sys.path.insert(0, "..")


In [ ]:
from helper_functions.helper_evaluate import compute_accuracy, compute_epoch_loss

from helper_functions.helper_data import *

from helper_functions.helper_train import *



ModuleNotFoundError: No module named 'helper_functions.helper_evaluate'

## 4. Dataset

In [ ]:
!pip install tqdm

In [ ]:
## Set random seed##
set_all_seeds(RANDOM_SEED)

### Dataset ###

train_transforms = transforms.Compose([
    transforms.Resize(70),
    transforms.RandomCrop(64, 64),
    transforms.ToTensor()
])

test_transforms = transforms.Compose([
    transforms.Resize(size=(70,70)),
    transforms.CenterCrop((64, 64)),
    transforms.ToTensor()
])

train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    num_workers=2,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    validation_fraction=0.1
)

In [ ]:
# Checking the dataset
print(f"Training set:\n")
for images, labels in train_loader:
  print(f"Image batch demensions: {images.size()}")
  print(f"Image label dimensions: {labels.size()}")
  print(labels[:10])
  break

# Checking the validation set
print(f"\nValidation set:")
for images, labels in valid_loader:
  print(f"Image batch demensions: {images.size()}")
  print(f"Image label dimensions: {labels.size()}")
  print(labels[:10])
  break

# Checking the testing set
print(f"\nTesting set:")
for images, labels in test_loader:
  print(f"Image batch demensions: {images.size()}")
  print(f"Image label dimensions: {labels.size()}")
  print(labels[:10])
  break

## 5. Model

In [ ]:
### MODEL ###

class AlexNet(nn.Module):
  def __init__(self, num_classes):
    super(AlexNet, self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),

      nn.Conv2d(64, 192, kernel_size=5, padding=2, groups=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),

      nn.Conv2d(192, 384, kernel_size=3, padding=1, groups=2),
      nn.ReLU(inplace=True),

      nn.Conv2d(384, 256, kernel_size=3, padding=1, groups=2),
      nn.ReLU(inplace=True),

      nn.Conv2d(256, 256, kernel_size=3, padding=1, groups=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
    self.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(256*6*6, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),
    )

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = x.view(x.size(0), 256*6*6)
    logits = self.classifier(x)
    probas = F.softmax(logits, dim=1)
    return logits

In [ ]:
torch.manual_seed(RANDOM_SEED)
model = AlexNet(NUM_CLASSES)
model = model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)